# Scene Landmarks Evaluation Notebook

This Jupyter Notebook provides a workflow for evaluating the association and clustering of landmarks detected in a scene with map features. The main steps include:

1. **Data Loading**: 
    - Loads scene-specific CSV files containing detected landmarks, candidate landmarks, and map features.

2. **Data Preparation**: 
    - Filters landmarks and map features for specific frames.
    - Adds appropriate headers to dataframes for clarity.

3. **Visualization**: 
    - Plots the spatial distribution of landmarks, candidates, and map features.
    - Highlights the region of interest using circles based on calculated centers and radii.

4. **Evaluation Functions**: 
    - Implements functions to calculate the center and maximum radius of landmarks.
    - Defines matching and evaluation logic using spatial queries (KD-Tree) to compare detected landmarks with map features.

5. **Frame-wise and Scene-wise Evaluation**: 
    - Evaluates matches for each frame and aggregates results.
    - Calculates precision, recall, and F1 scores for both clustering and association approaches.

6. **Results Summary**: 
    - Presents detailed and summary tables of evaluation metrics for easy comparison.

This notebook is useful for analyzing the performance of landmark detection and association algorithms in autonomous driving or mapping scenarios.

In [ ]:
import pprint
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import sys

In [ ]:
# Load the files
scene = "individual_files_validation_segment-10335539493577748957_1372_870_1392_870_with_camera_labels"
date = "202504141722"

# filepath of the csv file with the scene poses_data
landmarks_file_path = os.path.join(os.getcwd(), scene + "/" + date, "landmarks_" + scene + ".csv")
landmarks_candidates_file_path = os.path.join(os.getcwd(), scene + "/" + date, "landmarks_candidates_" + scene + ".csv")
print("Landmarks File Path:", landmarks_file_path)
print("Landmarks Candidates File Path:", landmarks_candidates_file_path)

In [ ]:
# Filepath of the signs map features csv file
signs_map_features_file_path = os.path.join(
    os.path.dirname((os.getcwd())),
    "pointcloud_clustering/map",
    "signs_map_features_" + scene + ".csv"
)
print("Signs Map Features File Path:", signs_map_features_file_path)

In [ ]:
# Load the CSV files into pandas DataFrames
landmarks_df = pd.read_csv(landmarks_file_path)
landmarks_candidates_df = pd.read_csv(landmarks_candidates_file_path)
signs_map_features_df = pd.read_csv(signs_map_features_file_path, header=None)

In [ ]:
landmarks_df

In [ ]:
landmarks_candidates_df

In [ ]:
# Add headers to the signs_map_features_df
signs_map_features_df.columns = ['Landmark_X', 'Landmark_Y', 'Landmark_Z']

In [ ]:
signs_map_features_df

In [ ]:
frame = 1
frame = '['+str(frame)+']'
filtered_landmarks_df = landmarks_df[landmarks_df['frame'] == frame]
filtered_landmarks_df

In [ ]:
filtered_landmarks_candidates_df = landmarks_candidates_df[landmarks_candidates_df['frame'] == frame]
filtered_landmarks_candidates_df

In [ ]:
def calculate_center_and_max_radius(landmarks_df):
    """
    Calcula el centro en X e Y y el radio máximo de los landmarks.

    Parameters:
        landmarks_df (pd.DataFrame): DataFrame que contiene las columnas 'Landmark_X' y 'Landmark_Y'.

    Returns:
        tuple: Una tupla con el centro en X, el centro en Y y el radio máximo.
    """
    if landmarks_df.empty:
        return None, None, None

    center_x = landmarks_df['Landmark_X'].mean()
    center_y = landmarks_df['Landmark_Y'].mean()

    # Calcular el radio máximo como la distancia máxima desde el centro
    distances = ((landmarks_df['Landmark_X'] - center_x)**2 + (landmarks_df['Landmark_Y'] - center_y)**2)**0.5
    max_radius = distances.max()

    return center_x, center_y, max_radius

In [ ]:
center_x, center_y, max_radius = calculate_center_and_max_radius(filtered_landmarks_candidates_df)
print(f"Centro en X: {center_x}, Centro en Y: {center_y}, Radio máximo: {max_radius}")

max_radius = max_radius + 10

In [ ]:
filtered_signs_map_features_df = signs_map_features_df[
        ((signs_map_features_df['Landmark_X'] - center_x)**2 + 
        (signs_map_features_df['Landmark_Y'] - center_y)**2)**0.5 <= max_radius
]

In [ ]:
# Draw the circle
circle = plt.Circle((center_x, center_y), max_radius, color='red', fill=False, linestyle='--', label=f'Radius Threshold: {max_radius:.2f} m')

# Plot the data
plt.figure(figsize=(10, 8))
plt.scatter(signs_map_features_df['Landmark_X'], signs_map_features_df['Landmark_Y'], color='blue', label='Landmarks (map)')
plt.scatter(filtered_signs_map_features_df['Landmark_X'], filtered_signs_map_features_df['Landmark_Y'], color='green', label=f'Landmarks (filtered for frame {frame})')
plt.scatter(filtered_landmarks_df['Landmark_X'], filtered_landmarks_df['Landmark_Y'], color='purple', label='Landmarks after association')
plt.scatter(filtered_landmarks_candidates_df['Landmark_X'], filtered_landmarks_candidates_df['Landmark_Y'], color='orange', label='Landmarks candidates')
plt.scatter(center_x, center_y, color='red', label='Center of the LiDAR area', marker='x')

# Add the circle to the plot
plt.gca().add_artist(circle)

# Add labels, legend, and grid
plt.legend()
plt.xlabel('X (m)')
plt.ylabel('Y (m)')
plt.grid()

# Set equal scaling for axes
plt.axis('equal')

plt.show()

In [ ]:
import scipy.spatial

# Función para evaluar los matches
def evaluate_matches(globalmapdata, evalmapdata):
    n = globalmapdata['poleparams'].shape[0]
    evalpolemap = evalmapdata['polemeans'][:, :2]
    n_eval = evalpolemap.shape[0]
    maxdist = 1.0
    kdtree = scipy.spatial.cKDTree(globalmapdata['poleparams'][:, :2], leafsize=10)
    dist, _ = kdtree.query(evalpolemap, k=1, distance_upper_bound=maxdist)
    n_matches = np.sum(np.isfinite(dist))

    matched_param = evalpolemap[np.isfinite(dist), :]
    TP = n_matches
    FP = n_eval - n_matches
    FN = n - n_matches
    precision = (TP + 0.0) / (TP + FP)
    recall = (TP + 0.0) / (TP + FN)
    F1 = 2 * precision * recall / (precision + recall)

    # Return matched_params as a dictionary
    matched_params = {
        'matched_coordinates': matched_param,
        'n_matches': n_matches,
        'TP': TP,
        'FP': FP,
        'FN': FN,
        'precision': precision,
        'recall': recall,
        'F1': F1
    }
    return matched_params

In [ ]:
def evaluate_frame(frame, landmarks_df, signs_map_df):
    """
    Evaluates matches for a specific frame.

    Parameters:
        frame (int): The frame number to evaluate.
        landmarks_df (pd.DataFrame): DataFrame containing landmarks data.
        signs_map_df (pd.DataFrame): DataFrame containing signs map features.

    Returns:
        dict: A dictionary containing matched parameters and evaluation metrics.
    """
    frame_candidates_df = landmarks_df[landmarks_df['frame'] == frame]

    # Calculate center and radius
    center_x, center_y, max_radius = calculate_center_and_max_radius(frame_candidates_df)
    if center_x is None or center_y is None or max_radius is None:
        return None

    # Filter signs map features using the center and radius
    filtered_signs_map = signs_map_df[
        ((signs_map_df['Landmark_X'] - center_x)**2 + 
         (signs_map_df['Landmark_Y'] - center_y)**2)**0.5 <= max_radius
    ]

    # Prepare data for evaluate_matches
    globalmapdata = {'poleparams': filtered_signs_map[['Landmark_X', 'Landmark_Y']].to_numpy()}
    evalmapdata = {'polemeans': frame_candidates_df[['Landmark_X', 'Landmark_Y']].to_numpy()}

    # Evaluate matches
    matched_params = evaluate_matches(globalmapdata, evalmapdata)

    return matched_params

In [ ]:
# Evaluate a single frame
clustering_frame_results = evaluate_frame('[1]', landmarks_candidates_df, signs_map_features_df)
pprint.pprint(clustering_frame_results)

In [ ]:
# Evaluate all frames
unique_frames = landmarks_candidates_df['frame'].unique()

results_clustering = []
results_association = []
for frame in unique_frames:
    results_clustering.append({'frame': frame, 'matched_params': evaluate_frame(frame, landmarks_candidates_df, signs_map_features_df)})
    results_association.append({'frame': frame, 'matched_params': evaluate_frame(frame, landmarks_df, signs_map_features_df)})

In [ ]:
# Create a DataFrame from the association results
results_association_df = pd.DataFrame(results_association)

# Expand the 'matched_params' column into separate columns, handling None values
results_association_df['matched_params'] = results_association_df['matched_params'].apply(lambda x: {} if x is None else x)
expanded_columns = results_association_df['matched_params'].apply(pd.Series)

# Concatenate the expanded columns with the original DataFrame
results_association_df = pd.concat([results_association_df.drop(columns=['matched_params']), expanded_columns], axis=1)

# Display the expanded DataFrame
results_association_df

In [ ]:
# Create a DataFrame from the association results
results_clustering_df = pd.DataFrame(results_clustering)

# Expand the 'matched_params' column into separate columns, handling None values
results_clustering_df['matched_params'] = results_clustering_df['matched_params'].apply(lambda x: {} if x is None else x)
expanded_columns = results_clustering_df['matched_params'].apply(pd.Series)

# Concatenate the expanded columns with the original DataFrame
results_clustering_df = pd.concat([results_clustering_df.drop(columns=['matched_params']), expanded_columns], axis=1)

# Display the expanded DataFrame
results_clustering_df

In [ ]:
# Merge clustering and association results on the 'frame' column
merged_results_df = pd.merge(
    results_clustering_df,
    results_association_df,
    on='frame',
    suffixes=('_clustering', '_association')
)

# Display the merged DataFrame
merged_results_df

In [ ]:
print("CLUSTERING RESULTS")

# Aggregate the results for the entire scene
total_clustering_TP = results_clustering_df['TP'].sum()
total_clustering_FP = results_clustering_df['FP'].sum()
total_clustering_FN = results_clustering_df['FN'].sum()

# Calculate overall precision, recall, and F1 score
overall_clustering_precision = total_clustering_TP / (total_clustering_TP + total_clustering_FP) if (total_clustering_TP + total_clustering_FP) > 0 else 0
overall_clustering_recall = total_clustering_TP / (total_clustering_TP + total_clustering_FN) if (total_clustering_TP + total_clustering_FN) > 0 else 0
overall_clustering_F1 = 2 * overall_clustering_precision * overall_clustering_recall / (overall_clustering_precision + overall_clustering_recall) if (overall_clustering_precision + overall_clustering_recall) > 0 else 0

print(f"Overall Precision: {overall_clustering_precision:.4f}")
print(f"Overall Recall: {overall_clustering_recall:.4f}")
print(f"Overall F1 Score: {overall_clustering_F1:.4f}")

In [ ]:
print("ASSOCIATION RESULTS")
# Aggregate the results for the entire scene
total_association_TP = results_association_df['TP'].sum()
total_association_FP = results_association_df['FP'].sum()
total_association_FN = results_association_df['FN'].sum()
# Calculate overall precision, recall, and F1 score
overall_association_precision = total_association_TP / (total_association_TP + total_association_FP) if (total_association_TP + total_association_FP) > 0 else 0
overall_association_recall = total_association_TP / (total_association_TP + total_association_FN) if (total_association_TP + total_association_FN) > 0 else 0
overall_association_F1 = 2 * overall_association_precision * overall_association_recall / (overall_association_precision + overall_association_recall) if (overall_association_precision + overall_association_recall) > 0 else 0
print(f"Overall Precision:  {overall_association_precision:.4f}")
print(f"Overall Recall:     {overall_association_recall:.4f}")
print(f"Overall F1 Score:   {overall_association_F1:.4f}")

In [ ]:
# Create a summary table
summary_data = {
    'Metric': ['Total TP', 'Total FP', 'Total FN', 'Overall Precision', 'Overall Recall', 'Overall F1 Score'],
    'Clustering': [
        total_clustering_TP,
        total_clustering_FP,
        total_clustering_FN,
        overall_clustering_precision,
        overall_clustering_recall,
        overall_clustering_F1
    ],
    'Association': [
        total_association_TP,
        total_association_FP,
        total_association_FN,
        overall_association_precision,
        overall_association_recall,
        overall_association_F1
    ]
}

summary_df = pd.DataFrame(summary_data)

# Display the summary table
summary_df